# Исследование надёжности заёмщиков

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

Результаты исследования будут учтены при построении модели **кредитного скоринга** — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

## Шаг 1. Откройте файл с данными и изучите общую информацию

### Обзор данных

Основной инструмент аналитика — `pandas`. Импортируйте эту библиотеку.

In [1]:
import pandas as pd    #импорт библиотеки pandas как pd

In [2]:
data = pd.read_csv('data.csv')    # чтение файла с данными и сохранение в data

In [3]:
data.head(30)    # получение первых 15 строк таблицы data

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


In [4]:
data.info()    # получение общей информации о данных в таблице data

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


In [5]:
data.describe() #применение метода describe()

,children,days_employed,dob_years,education_id,family_status_id,debt,total_income
count,21525.000000,19351.000000,21525.000000,21525.000000,21525.000000,21525.000000,1.935100e+04
mean,0.538908,63046.497661,43.293380,0.817236,0.972544,0.080883,1.674223e+05
std,1.381587,140827.311974,12.574584,0.548138,1.420324,0.272661,1.029716e+05
min,-1.000000,-18388.949901,0.000000,0.000000,0.000000,0.000000,2.066726e+04
25%,0.000000,-2747.423625,33.000000,1.000000,0.000000,0.000000,1.030532e+05
50%,0.000000,-1203.369529,42.000000,1.000000,0.000000,0.000000,1.450179e+05
75%,1.000000,-291.095954,53.000000,1.000000,1.000000,0.000000,2.034351e+05
max,20.000000,401755.400475,75.000000,4.000000,4.000000,1.000000,2.265604e+06


**Вывод**

В таблице 12 столбцов, в некоторых столбцах тип данных — `object`, в других - `float64` и `int64`

Согласно документации к данным:
* `children` — количество детей в семье;
* `days_employed` — общий трудовой стаж в днях;  
* `dob_years` — возраст клиента в годах;
* `education` — уровень образования клиента;
* `yeducation_id` — идентификатор уровня образования;
* `family_status` — семейное положение;
* `family_status_id` — идентификатор семейного положения.
* `gender` — пол клиента;
* `income_type` — тип занятости;  
* `debt` — имел ли задолженность по возврату кредитов;
* `total_income` — ежемесячный доход;
* `purpose` — цель получения кредита;

В колонке `education` разный регистр, в колонке `days_employed` встречаются отрицательные значения.
Количество значений в столбцах различается. Значит, в данных есть пропущенные значения.

## Шаг 2. Предобработка данных

### Обработка пропусков

Посчитаем количество пропусков в каждом столбце методом isna().

In [6]:
data.isna().sum()

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

Обнаружены пропуски в колонках days_employed и total_income, это колличественные переменные и пропуски в таких переменных заполняются характерными значениями. Причиной возникновения пропусков может служить не раскрытие данных о доходе и стаже госслужащих, так же отсутствуют данные о пенсионерах в виду того что отсутствовал цифровой сбор информации в период их трудовой деятельности.

Заполним пропуски в колонке total_income и days_employed применением метода fillna(). Пропуски в days_employed заполним средним значением, а в total_income медианным значением, т.к. некоторые значения сильно выделяются. Медианное значение будет браться по данным сгрупированным по столбцу income_type, т.к. каждому типу занятости соответствует разный доход.

In [7]:
data['days_employed'] = data['days_employed'].abs()

In [8]:
data['days_employed'] = data['days_employed'].fillna(data.groupby('income_type')['days_employed'].transform('mean'))


<div class="alert alert-success">
<b>✔️ Комментарий ревьюера:</b> 
<br>Отличная работа! Хоть этот столбец нам и не нужен для ответа на итоговые вопросы, но круто, что ты всё равно решил с ним поработать.
</div>

In [9]:
data['total_income'] = data['total_income'].fillna(data.groupby('income_type')['total_income'].transform('median'))

<div class="alert alert-success">
<b>✔️ Комментарий ревьюера:</b> 
<br>То что нужно! Это отличный способ для замены пропусков в данном случае.
</div>

После заполнения проверяем на пропуски.

In [10]:
data.isna().sum()

children            0
days_employed       0
dob_years           0
education           0
education_id        0
family_status       0
family_status_id    0
gender              0
income_type         0
debt                0
total_income        0
purpose             0
dtype: int64

**Вывод**

Пропусков в данных было 10%, это много и могло повлиять на выявление зависимостей.

### Замена типа данных

Заменим тип данных в столбцах days_employed и total_income методом astype().

In [11]:
data['days_employed'] = data['days_employed'].astype('int')

In [12]:
data['total_income'] = data['total_income'].astype('int')

Проверим тип данных.

In [13]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   children          21525 non-null  int64 
 1   days_employed     21525 non-null  int64 
 2   dob_years         21525 non-null  int64 
 3   education         21525 non-null  object
 4   education_id      21525 non-null  int64 
 5   family_status     21525 non-null  object
 6   family_status_id  21525 non-null  int64 
 7   gender            21525 non-null  object
 8   income_type       21525 non-null  object
 9   debt              21525 non-null  int64 
 10  total_income      21525 non-null  int64 
 11  purpose           21525 non-null  object
dtypes: int64(7), object(5)
memory usage: 2.0+ MB


Тип данных float64 изменен на int64.

Приведем данные в столбце days_employed к единице измерения "года".

In [14]:
data['days_employed'] = data['days_employed'] / 365    #деление количества дней стажа на 365, получение таким образом стажа в годах

Приведем данные в столбце total_income к единице измерения "тысяч рублей".

In [15]:
data['total_income'] = data['total_income'] / 1000    #деление ежемесячный доход на 1000, получение значений в тысячах рублей

Приведем данные в столбце education к нижнему регистру.

In [16]:
data['education'] = data['education'].str.lower()

Проверим изменеение регистра.

In [17]:
data.head(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,23.115068,42,высшее,0,женат / замужем,0,F,сотрудник,0,253.875,покупка жилья
1,1,11.024658,36,среднее,1,женат / замужем,0,F,сотрудник,0,112.080,приобретение автомобиля
2,0,15.405479,33,среднее,1,женат / замужем,0,M,сотрудник,0,145.885,покупка жилья
3,3,11.298630,32,среднее,1,женат / замужем,0,M,сотрудник,0,267.628,дополнительное образование
4,0,932.235616,53,среднее,1,гражданский брак,1,F,пенсионер,0,158.616,сыграть свадьбу
5,0,2.536986,27,высшее,0,гражданский брак,1,M,компаньон,0,255.763,покупка жилья
6,0,7.887671,43,высшее,0,женат / замужем,0,F,компаньон,0,240.525,операции с жильем
7,0,0.416438,50,среднее,1,женат / замужем,0,M,сотрудник,0,135.823,образование
8,2,18.983562,35,высшее,0,гражданский брак,1,F,сотрудник,0,95.856,на проведение свадьбы
9,0,5.994521,41,среднее,1,женат / замужем,0,M,сотрудник,0,144.425,покупка жилья для семьи


**Вывод**

Изменен тип данных для применения к ним математических операций, так же значения приведены к нижнему регистру что позволит выделять категории и избежать дубликатов.

### Обработка дубликатов

Применим метод duplicated() к датасету чтобы выявить явные дубликаты.

In [18]:
data.duplicated().sum()

71

Количество дубликатов не большое, удалим их методом drop_duplicates().

In [19]:
data = data.drop_duplicates().reset_index(drop=True)

Проверка на дубликаты.

In [20]:
data.duplicated().sum()

0

**Вывод** 

Удалили дубликаты с сохранением индексации для дальнейшего исследования.

### Лемматизация

Цели кредита все разные, но бывают совпадения, применим метод лемматизации библиоеки pymystem3? чтобы выделить леммы.

In [21]:
from pymystem3 import Mystem
m = Mystem()
data['lemma_purpose'] = data['purpose'].apply(m.lemmatize)

In [22]:
data['lemma_purpose'].value_counts()

TypeError: unhashable type: 'list'

Exception ignored in: 'pandas._libs.index.IndexEngine._call_map_locations'
Traceback (most recent call last):
  File "pandas/_libs/hashtable_class_helper.pxi", line 4588, in pandas._libs.hashtable.PyObjectHashTable.map_locations
TypeError: unhashable type: 'list'


[автомобиль, \n]                                          972
[свадьба, \n]                                             791
[на,  , проведение,  , свадьба, \n]                       768
[сыграть,  , свадьба, \n]                                 765
[операция,  , с,  , недвижимость, \n]                     675
[покупка,  , коммерческий,  , недвижимость, \n]           661
[операция,  , с,  , жилье, \n]                            652
[покупка,  , жилье,  , для,  , сдача, \n]                 651
[операция,  , с,  , коммерческий,  , недвижимость, \n]    650
[покупка,  , жилье, \n]                                   646
[жилье, \n]                                               646
[покупка,  , жилье,  , для,  , семья, \n]                 638
[строительство,  , собственный,  , недвижимость, \n]      635
[недвижимость, \n]                                        633
[операция,  , со,  , свой,  , недвижимость, \n]           627
[строительство,  , жилой,  , недвижимость, \n]            624
[покупка

**Вывод** 

С помощью лемматизации выявили основные леммы: автомобиль, жилье, недвижимость, свадьба, образование. По этим леммам будем категоризировать данные.

### Категоризация данных

Категоризируем данные, для дальнейшей группиворки их по целям кредита. Для этого применим функцию для одной строки.

In [23]:
def purpose_credit(row):
    
    lemma_purpose = row['lemma_purpose']
    
    if 'автомобиль' in lemma_purpose:
        return 'автомобиль'
    if 'свадьба' in lemma_purpose:
        return 'свадьба'
    if ('жилье' in lemma_purpose) or ('недвижимость' in lemma_purpose):
        return 'недвижимость'
    if 'образование' in lemma_purpose:
        return 'образование'
    
data['cat_purpose'] = data.apply(purpose_credit, axis=1)


Проверим результат.

In [24]:
data['cat_purpose'].head(15)

0     недвижимость
1       автомобиль
2     недвижимость
3      образование
4          свадьба
5     недвижимость
6     недвижимость
7      образование
8          свадьба
9     недвижимость
10    недвижимость
11    недвижимость
12         свадьба
13      автомобиль
14    недвижимость
Name: cat_purpose, dtype: object

**Вывод**

Благодаря категоризации данных получили новый столбец, с помощью которого сможем оценить зависимоть как разные цели кредита влияют на его возврат в срок.

## Шаг 3. Ответьте на вопросы

- Есть ли зависимость между наличием детей и возвратом кредита в срок?

Посчитаем зависимость по двум категориям с детьси и без детей.

In [25]:
data['children'].value_counts()

 0     14091
 1      4808
 2      2052
 3       330
 20       76
-1        47
 4        41
 5         9
Name: children, dtype: int64

В данных обнаружено что 47 человек имеют минус одного ребенка, осправим методом abs(), который возвращает модуль числа.

In [26]:
data['children'] = data['children'].abs()

In [27]:
children_debt = data[['children', 'debt']]

no_one_children = children_debt[children_debt['children'] == 0]['children'].count()    # колическтво людей без детей
no_one_children_convr = children_debt[(children_debt['children'] == 0) & (children_debt['debt'] == 1)]['children'].count() / no_one_children    # конверсия по категории без детей

several_children = children_debt[children_debt['children'] > 0]['children'].count()    # количество людей с детьми
several_children_convr = children_debt[(children_debt['children'] > 0) & (children_debt['debt'] == 1)]['children'].count() / several_children    # конверсия по категории с детьми

In [28]:
display('Процент просрочки по кредиту категории без детей : {:.2%}'.format(no_one_children_convr))
display('Процент просрочки по кредиту категории с детьми : {:.2%}'.format(several_children_convr))

'Процент просрочки по кредиту категории без детей : 7.54%'

'Процент просрочки по кредиту категории с детьми : 9.21%'

Аналогичный метод.

In [29]:
children_debt_group = data.groupby('children')['debt'].mean()*100
children_debt_group

children
0      7.543822
1      9.165808
2      9.454191
3      8.181818
4      9.756098
5      0.000000
20    10.526316
Name: debt, dtype: float64

**Вывод** 

Категория людей с детьми имеет больше просрочек по кредиту, что логично исходя из того что у них больше потребностей. Так же в данной выборке у людей с 5 детьми нет просрочек вообще.

- Есть ли зависимость между семейным положением и возвратом кредита в срок?

Выведем основные категории колонки family_status.

In [30]:
data['family_status'].unique()

array(['женат / замужем', 'гражданский брак', 'вдовец / вдова',
       'в разводе', 'Не женат / не замужем'], dtype=object)

Получили 5 основных категории по которым и будем проводить расчет.

In [31]:
family_debt = data[['family_status', 'debt']]

married = family_debt[(family_debt['family_status'] == 'женат / замужем')]['family_status'].count()
married_convr = family_debt[(family_debt['family_status'] == 'женат / замужем') & (family_debt['debt'] == 1)]['family_status'].count() / married

no_married = family_debt[(family_debt['family_status'] == 'Не женат / не замужем')]['family_status'].count() 
no_married_convr = family_debt[(family_debt['family_status'] == 'Не женат / не замужем') & (family_debt['debt'] == 1)]['family_status'].count() / no_married

civil_marriage = family_debt[(family_debt['family_status'] == 'гражданский брак')]['family_status'].count()
civil_marriage_convr = family_debt[(family_debt['family_status'] == 'гражданский брак') & (family_debt['debt'] == 1)]['family_status'].count() / civil_marriage

divorced = family_debt[(family_debt['family_status'] == 'в разводе')]['family_status'].count() 
divorced_convr = family_debt[(family_debt['family_status'] == 'в разводе') & (family_debt['debt'] == 1)]['family_status'].count() / divorced

widow = family_debt[(family_debt['family_status'] == 'вдовец / вдова')]['family_status'].count() 
widow_convr = family_debt[(family_debt['family_status'] == 'вдовец / вдова') & (family_debt['debt'] == 1)]['family_status'].count() / widow

In [32]:
display('Процент просрочки по кредиту категории женат / замужем : {:.2%}'.format(married_convr))
display('Процент просрочки по кредиту категории не женат / не замужем : {:.2%}'.format(no_married_convr))
display('Процент просрочки по кредиту категории гражданский брак : {:.2%}'.format(civil_marriage_convr))
display('Процент просрочки по кредиту категории в разводе : {:.2%}'.format(divorced_convr))
display('Процент просрочки по кредиту категории в вдовец / вдова : {:.2%}'.format(widow_convr))

'Процент просрочки по кредиту категории женат / замужем : 7.55%'

'Процент просрочки по кредиту категории не женат / не замужем : 9.75%'

'Процент просрочки по кредиту категории гражданский брак : 9.35%'

'Процент просрочки по кредиту категории в разводе : 7.11%'

'Процент просрочки по кредиту категории в вдовец / вдова : 6.57%'

Выведем сводную таблицу, методом pivot_table() 

In [33]:
data_pivot = data.pivot_table(index=['family_status'], columns='debt', values='family_status_id', aggfunc='count')
data_pivot['% просрочки'] = (data_pivot[1] / (data_pivot[0] + data_pivot[1]))*100

In [34]:
data_pivot.style.format({'% просрочки':'{:.2f}%'})

debt,0,1,% просрочки
family_status,,,
Не женат / не замужем,2536,274,9.75%
в разводе,1110,85,7.11%
вдовец / вдова,896,63,6.57%
гражданский брак,3763,388,9.35%
женат / замужем,11408,931,7.55%


Из расчетов следует, что семейное положение влияет на своевременное покрытие долговых обязательств. Ответственнее всего к погашению кредита относятся вдовец / вдова,обычно это взрослые люди и в силу своего возраста лучше контролируют свои долговые обязательства. Дальше идет категория людей в разводе, которые в силу обстоятельств хорошо следят за долговыми обязательствами. Далее категория женатых людей которые платят по долгам лучше людей категорий гражданский брак и Не женат / не замужем в силу своего семейного бюджета, который складывается из доходов обоих членов семьи.

- Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

Введем категории достатка.
* `250 >` — низкий доход;
* `500 > и > 250` — средний доход;  
* `500 <` — высокий доход;

In [35]:
mean = 250
high = 500

Найдем просрочки по кредиту по уровню достатка.

In [36]:
income_debt = data[['total_income','debt']]
low_income = income_debt[income_debt['total_income'] < mean]['total_income'].count() # с низким доходом
low_income_convr = income_debt[(income_debt['total_income'] < mean) & (income_debt['debt'] == 1)]['total_income'].count() / low_income

average_income = income_debt[(income_debt['total_income'] > mean) & (income_debt['total_income'] < high)]['total_income'].count() # со средним доходом
average_income_convr = income_debt[(income_debt['total_income'] > mean) & (income_debt['total_income'] < high) & (income_debt['debt'] == 1)]['total_income'].count() / average_income

high_income = income_debt[income_debt['total_income'] > high]['total_income'].count() # с высоким доходом
high_income_convr = income_debt[(income_debt['total_income'] > high) & (income_debt['debt'] == 1)]['total_income'].count() / high_income

In [37]:
display('Процент просрочки по кредиту категории с низким доходом : {:.2%}'.format(low_income_convr))
display('Процент просрочки по кредиту категории с средним доходом : {:.2%}'.format(average_income_convr))
display('Процент просрочки по кредиту категории с высоким доходом : {:.2%}'.format(high_income_convr))

'Процент просрочки по кредиту категории с низким доходом : 8.30%'

'Процент просрочки по кредиту категории с средним доходом : 6.95%'

'Процент просрочки по кредиту категории с высоким доходом : 6.31%'

**Вывод**

Из расчетов следует линейная зависимость уровня заработки и покрытие долговых обязательств, чем больше доход тем меньше процент просрочки по кредиту.

- Как разные цели кредита влияют на его возврат в срок?

Выделим основные цели кредита.

In [38]:
data['cat_purpose'].unique()

array(['недвижимость', 'автомобиль', 'образование', 'свадьба'],
      dtype=object)

Найдем просрочки по кредитам по цели кредита.

In [39]:
purpose_debt = data[['cat_purpose', 'debt']]

purpose_house = purpose_debt[purpose_debt['cat_purpose'] == 'недвижимость']['cat_purpose'].count()
purpose_house_convr = purpose_debt[(purpose_debt['cat_purpose'] == 'недвижимость') & (purpose_debt['debt'] == 1)]['cat_purpose'].count() / purpose_house

purpose_avto = purpose_debt[purpose_debt['cat_purpose'] == 'автомобиль']['cat_purpose'].count()
purpose_avto_convr = purpose_debt[(purpose_debt['cat_purpose'] == 'автомобиль') & (purpose_debt['debt'] == 1)]['cat_purpose'].count() / purpose_avto

purpose_education = purpose_debt[purpose_debt['cat_purpose'] == 'образование']['cat_purpose'].count()
purpose_education_convr = purpose_debt[(purpose_debt['cat_purpose'] == 'образование') & (purpose_debt['debt'] == 1)]['cat_purpose'].count() / purpose_education

purpose_wedding = purpose_debt[purpose_debt['cat_purpose'] == 'свадьба']['cat_purpose'].count()
purpose_wedding_convr = purpose_debt[(purpose_debt['cat_purpose'] == 'свадьба') & (purpose_debt['debt'] == 1)]['cat_purpose'].count() / purpose_wedding

In [40]:
display('Процент просрочки с целью кредита недвижимость : {:.2%}'.format(purpose_house_convr))
display('Процент просрочки с целью кредита автомобиль : {:.2%}'.format(purpose_avto_convr))
display('Процент просрочки с целью кредита образование : {:.2%}'.format(purpose_education_convr))
display('Процент просрочки с целью кредита свадьба : {:.2%}'.format(purpose_wedding_convr))

'Процент просрочки с целью кредита недвижимость : 7.23%'

'Процент просрочки с целью кредита автомобиль : 9.36%'

'Процент просрочки с целью кредита образование : 9.22%'

'Процент просрочки с целью кредита свадьба : 8.00%'

Из расчетов следует, что долговые обязательства за недвижимость, это важнее всего, так же кредит на свадьбу оказалася важен кредита на автомобиль и образование.

## Шаг 4. Общий вывод

По результатам исследования выявлено следующее:
* наличие детей влияет на покрытие долговых обязательств, люди без детей платят лучше.
* семейное положение влияет на покрытие долговых обязательств, вдовцы лучше платят по долговым обязательствам.
* линейная зависимость уровня дохода и покрытие долговых обязательств, чем больше доход тем меньше просрочек по кредиту.
* цель кредита так же важна, за недвижимость и свадьбу люди платят охотнее чем за автомобиль и образование.